# Project Description

**The Question**: Which features of public schools predict above-average school ranking?

This project uses data that I purchased and loaded from the GreatSchools.org API in 2021. The API endpoint for school rankings and demographics data was defined by a radius of geographical distance from Mount Kisco, NY, where I lived at the time. 

In all likelihood, I will purchase more data from GreatSchools to perform a 4-year comparison.


**What the GreatSchools Rating measures:**

The GreatSchools Rating is a 1–10 score, purchased by real estate listing aggregators like Zillow. Ratings at the lower end of the scale (1–4) signal below-average performance, 5–6 indicate average performance, and 7–10 demonstrate above-average performance.

The GreatSchools Rating is based on up to three themed ratings, which are designed to capture different aspects of school quality:

1) Student Progress Rating: Calculated using state-reported student growth data. If unavailable, it is replaced with the Academic Progress Rating, a proxy growth measure GreatSchools creates when sufficient state-produced student growth data is not available.
2) College Readiness Rating: A multi-measure rating based on college entrance exams (SAT and ACT), high school graduation rates, and advanced coursework participation (Advanced Placement (AP), International Baccalaureate (IB), or dual enrollment).
3) Test Score Rating: Based on state-standardized test performance.

GreatSchools calculate the 1-10 Rating using a two-step weighted average approach.
1) First, they assign base weights to each themed rating based on the strength of research linking that data to long-term student outcomes and our mission to highlight schools that support academic growth for all students. 
2) Second, they apply information weights that reflect the quantity and variability of data available for each themed rating. To maintain balance, these weights are capped so that no themed rating can outweigh the Student Progress Rating (or the Academic Progress Rating if growth data is not available). If any themed ratings are missing, the remaining weights are rebalanced to sum to one.

For more detail, see: https://www.greatschools.org/gk/about/ratings-methodology/


# Imports Table

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Exponential notation that preserves float up to 4 deep
def smart_num_format(x):
    if abs(x) < 9_999_999:
        return f"{x:.4f}"
    else:
        return f"{x:.4e}"
pd.set_option('display.float_format', smart_num_format)


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split


from sklearn.preprocessing import StandardScaler

from sklearn.base import BaseEstimator, TransformerMixin # for ILR transform

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression



from xgboost import XGBClassifier

import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_curve, auc,
    precision_recall_curve,
    average_precision_score
)
from sklearn.calibration import calibration_curve

NameError: name 'pd' is not defined

# EDA

Data consists of two tables, which I constructed by parsing the JSON emitted from the GreatSchools.org API.

1) schools_raw - Describes location, grade level, rating, and distance to origin
- 2,349 schools
- id (joined to Universal_ID in demographics)
- 13 feature columns



2) demog_raw - Describes enrollment, teacher salaries, student-teacher ratio, socioeconomic class markers, racial composition
- 1,598 schools
- Universal_ID
- 18 feature columns



In [ ]:
# read in schools raw data
schools_raw = pd.read_csv('schools_data_BACKUP.csv')
schools = schools_raw.copy()

In [ ]:
schools.head(2)

In [ ]:
schools.shape

In [ ]:
# read in demographics raw data
demog_raw = pd.read_csv('demographics_mstr_BACKUP.csv')
demog = demog_raw.copy()

In [ ]:
demog.shape

In [ ]:
demog.head(2)

In [ ]:
# merge right on 'id', left on 'Universal_ID'

df = schools.merge(demog, left_on='id', right_on='Universal_ID', how='outer')

In [ ]:
# inspect merge results

df.head()

In [ ]:
# look for big patterns

df[df.columns.difference(['id','rating_year', 'Universal_ID'])].describe().T

In [ ]:
# observe distribution of school ratings

df['rating'].hist()
plt.title('Distribution of School Ratings')

In [ ]:
# check for duplicates in main identifier

df[df.duplicated(subset='id', keep=False)]

In [ ]:
# schools per district

df['district'].value_counts()

In [ ]:
# mean rating by district

(
    df.groupby(by='district')['rating']
    .agg(mean='mean').sort_values(by='mean', ascending=False).head(50)
)



In [ ]:
# missingness

sns.heatmap(df.isnull())
plt.title('Missingness')
plt.show()

In [ ]:
# percent null

(df.isnull().sum()/len(df)*100).sort_values(ascending=False)

In [ ]:
# drop 100% null column

df = df.drop(columns=['percent-disadvantaged'])

In [ ]:
# cross join on district to fill average salary?
# check if each school district has one unique "average salary"

unique_salary = (
    df.groupby(by='district')['average-salary']
    .agg(nunique='nunique').sort_values(by='nunique')['nunique'].eq(1)
)

unique_salary

In [ ]:
#inspect salaries per district
df.dropna(subset='district').set_index(['district','average-salary']).head()

In [ ]:
# observe asian and 'Asian or Pacific Islander'

df[['Asian', 'Asian or Pacific Islander']] # appears to be an or-condition

bad_overlap = (df['Asian'].notna() & df['Asian or Pacific Islander'].notna())

assert not bad_overlap.any(), 'Columns not mergable in current form'

In [ ]:
# merge asian and asian or pacific islander

df['Asian_combo'] = (
    df['Asian'].fillna(df['Asian or Pacific Islander'])
)

# drop original cols

df = df.drop(columns=['Asian', 'Asian or Pacific Islander'])


In [ ]:
# inspect feature correlations with heatmap

fig, ax = plt.subplots(figsize=(16,16))
corr = df.drop(
    columns=['Universal_ID','id','zip_code','distance', 'rating_year','latitude', 'longitude']
).select_dtypes(include='number').corr()
sns.heatmap(corr, annot=True, cbar=False, cmap='coolwarm')
plt.show()

In [ ]:
# drop data with no rating

df_rating = df.dropna(subset='rating').copy()

In [ ]:
# check missingness after dropping blank ratings

sns.heatmap(df_rating.isna(), cbar=False)
plt.title('Missingness After Dropna on Rating')
plt.show()

In [ ]:
#observe remaining missingness

(df_rating.isnull().sum()/len(df_rating)*100).sort_values(ascending=False)

In [ ]:
# stacked bar chart

demog_race = ['Hispanic', 'White','African American', 'Asian_combo', 
              'Native Hawaiian or Other Pacific Islander', 'Two or more races',
              'Native American']

plot_df = (
    df_rating
    .groupby('rating')[demog_race]
    .mean()     # or .sum(), depending on meaning
    .sort_index()
)

ax = plot_df.plot(kind='bar', stacked=True, figsize=(8, 5))
ax.set_ylabel('Share')
ax.set_xlabel('Rating')
ax.legend(title='Race', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title("NYC Metro Demographic Composition by Public School Ranking")
plt.tight_layout()


In [ ]:
# model funding-related features


bar_cols = ['percent-reduced-lunch', 
'student-counselor-ratio', 
'teachers-experience', 
'percentage-certified', 
'average-salary',
           'student-teacher-ratio']

for col in bar_cols:
    plot_table = (df_rating.groupby('rating')[col]
    .agg(col='mean')
    .sort_index())
    
    plot_table.plot(kind='bar', legend=False)
    plt.title(f'{col.title().replace("-"," ")} by Rating')
    plt.ylabel(f"{col}".replace("-"," "))
    plt.show()

In [ ]:
# check_for_outliers

scaler = StandardScaler() #apples to apples
scaled_array = scaler.fit_transform(df_rating.select_dtypes(include=['number']).copy())
df_scaled = pd.DataFrame(scaled_array,
                        columns=df_rating.select_dtypes(include=['number']).columns, 
                         index=df_rating.index)

df_scaled
sns.boxplot(df_scaled, orient='h')
plt.show()



In [ ]:
# define simplex cols

simplex_cols = ['Hispanic', 'White',
       'African American', 'Native Hawaiian or Other Pacific Islander',
       'Two or more races', 'Native American', 'Asian_combo']


In [ ]:
#check if any cols add to 100

#assert np.isclose(df_rating[simplex_cols].sum(axis=1), 100).all(), "Simplexes not all mathing"

In [ ]:
# rows where simplex columns are working

sns.heatmap(df_rating[np.isclose(df_rating[simplex_cols].sum(axis=1), 100, atol=0.01)].isna(), cbar=False)
plt.title('Missingness Where Simplex Cols Add to One')
plt.show()

In [ ]:
# observe rows where simplexes do not add up

df_rating[~np.isclose(df_rating[['Hispanic', 'White',
       'African American', 'Native Hawaiian or Other Pacific Islander',
       'Two or more races', 'Native American', 'Asian_combo']].sum(axis=1), 100, atol=0.01)]['state'].value_counts()

# all rows are in the state of Connecticut, which maybe does not publish demographic data

In [ ]:
# drop rows where state is connecticut
# connecticut data could be useful in some other context

ct_rows = df_rating[df_rating['state']=='CT'].index

df_rating = df_rating.drop(ct_rows)

In [ ]:
# verify that simplex columns now add all to one with Connecticut dropped
assert np.isclose(df_rating[['Hispanic', 'White',
       'African American', 'Native Hawaiian or Other Pacific Islander',
       'Two or more races', 'Native American', 'Asian_combo']].sum(axis=1), 100).all(), "Simplexes not mathing"

In [ ]:
# fill null values in add-to-one columns with zeros
# if the columns add to one, the nulls represent zeros

df_rating[simplex_cols] = df_rating[simplex_cols].fillna(0.0)

## Changes Log

Dropped:
1) The "percent-disadvantaged" column, which was 100% blank
2) Rows (schools) where "rating" column was blank
3) All rows for the state of Connecticut, which lacked demographic data

In [ ]:
# check missingness again
sns.heatmap(df_rating.isnull())
plt.show()

In [ ]:
# observe nulls
# could these be zeros in disguise?
nulls_cols = df_rating.columns[df_rating.isnull().sum()>0]

df_rating[nulls_cols]

# Pipeline

I use sklearn Pipeline to manage preprocessing, modelling, cross-validation

In [ ]:
# construct a binary target
mean_rating = df_rating['rating'].mean()

df_rating['above_average'] = np.where(df_rating['rating']>mean_rating, 1, 0)

In [ ]:
# condense zip codes into smaller categories

df_rating['zip_4'] = df_rating['zip_code'].astype('str').str[0:4]

In [ ]:
# create ILR transformer
class ILRTransformer(BaseEstimator, TransformerMixin):
    """
    Apply ILR transform to a set of compositional columns.
    X is expected to be a DataFrame when used standalone;
    inside ColumnTransformer it will be a NumPy array with only `cols`.
    """
    def __init__(self, cols, prefix="ilr"):
        self.cols = cols
        self.prefix = prefix
        self.V_ = None   # ILR basis (D-1 x D)
        self.feature_names_out_ = None

    def fit(self, X, y=None):
        D = len(self.cols)
        # Build a Helmert-like matrix to get an orthonormal basis
        H = np.zeros((D, D))
        for i in range(1, D):
            H[i, :i] = 1.0 / i
            H[i, i] = -1.0
        # QR on the (D x (D-1)) submatrix to get orthonormal columns
        Q, _ = np.linalg.qr(H[:, 1:])
        self.V_ = Q.T      # shape: (D-1, D)

        # tell sklearn how many features we output and what they’re called
        n_out = D - 1
        self.feature_names_out_ = np.array(
            [f"{self.prefix}_{j}" for j in range(n_out)],
            dtype=object
        )
        return self

    def transform(self, X):
        # handle both DataFrame and ndarray (inside ColumnTransformer)
        if isinstance(X, pd.DataFrame):
            Xs = X[self.cols].to_numpy(dtype=float)
        else:
            Xs = np.asarray(X, dtype=float)

        # Avoid log(0)
        Xs = np.clip(Xs, 1e-12, None)
        logX = np.log(Xs)
        clr = logX - logX.mean(axis=1, keepdims=True)  # n x D
        # ILR coords: n x (D-1)
        Z = clr @ self.V_.T
        return Z

    def get_feature_names_out(self, input_features=None):
        # ColumnTransformer will pass in the original feature names subset;
        # we just ignore and return our ILR coord names.
        return self.feature_names_out_


In [ ]:
# pipeline

# define the columns
simplex_cols = simplex_cols

num_cols     = ['distance', 'enrollment', 'percent-reduced-lunch',
       'percent-limited-english', 'average-salary', 'student-teacher-ratio',
       'student-counselor-ratio', 'percentage-female', 'teachers-experience',
       'percentage-certified']   

cat_cols     = ["zip_4", 'state']   # not many states represented in the data


# numeric pipeline: impute + scale

cat_pre = Pipeline([
    ("encode", OneHotEncoder(handle_unknown='ignore'))
    
])

num_pre = Pipeline([
    ("impute", SimpleImputer(strategy="mean", add_indicator=True)),
    ("scale", StandardScaler()),
])

ilr_pre = Pipeline([
    ("ilr", ILRTransformer(simplex_cols)),
    ("scale", StandardScaler())
])


logit_preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pre, num_cols),
        ("cat", cat_pre, cat_cols),
        ("ilr", ilr_pre, simplex_cols)
    ],
    remainder="drop",
)

tree_preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pre, num_cols),
        ("cat", cat_pre, cat_cols),
        ("simplex", "passthrough", simplex_cols),
    ],
    remainder="drop",
)


logit_pipe = Pipeline([
    ("preprocess", logit_preprocess),
    ("logit", LogisticRegression(max_iter=2000))
])

tree_pipe = Pipeline([
    ("preprocess", tree_preprocess),
    ('tree', DecisionTreeClassifier())
])

xgb_pipe = Pipeline([
    ("preprocess", tree_preprocess),
    ("xgb", XGBClassifier(random_state=21))
])



In [ ]:
# split the data
X = df_rating[simplex_cols + cat_cols + num_cols]
y = df_rating['above_average']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [ ]:
# decision tree model hyperparameter tuning


param_grid = {
    "tree__max_depth": [None, 3, 5, 10],
    "tree__min_samples_leaf": [1, 5, 10, 20, 30],
    "tree__min_samples_split": [2, 5, 10]
}

grid = GridSearchCV(
    tree_pipe,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
)

grid.fit(X_train, y_train)

grid.best_params_, grid.best_score_

In [ ]:
# tune the hyperparameters

from sklearn.model_selection import GridSearchCV

xgb_param_grid = {
    "xgb__n_estimators": [100, 300],
    "xgb__max_depth": [3, 5],
    "xgb__learning_rate": [0.05, 0.1],
}

xgb_gs = GridSearchCV(
    xgb_pipe,
    param_grid=xgb_param_grid,
    cv=5,
    scoring="accuracy",   # or whatever you’re using
    n_jobs=-1
)

xgb_gs.fit(X_train, y_train)

xgb_gs.best_params_, xgb_gs.best_score_


In [ ]:
# redefine tree_pipe

best_tree_pipe = grid.best_estimator_

best_xgb_pipe = xgb_gs.best_estimator_

In [ ]:
# keep pipes in a list

model_pipes_list = [("logit", logit_pipe), ("tree", best_tree_pipe), ('xgboost', best_xgb_pipe)]

In [ ]:
# pipeline

score_table = []

for name, pipe in model_pipes_list:
    scores = cross_val_score(
        pipe,
        X_train,
        y_train,
        cv=25,
        scoring="accuracy"   
    )
    
    score_table.append({
        "model": name,
        "mean_train_score": scores.mean(),
        "cv_scores": scores
    })

pd.DataFrame(score_table)


In [ ]:
# test scores

test_scores = []

for name, pipe in model_pipes_list:
    
    pipe.fit(X_train, y_train)
    
    test_scores.append({
        "model" : name,
        "num_features" : len(pipe.named_steps['preprocess'].get_feature_names_out()),
        "test_score" : pipe.score(X_test, y_test),
        
    })
    
pd.DataFrame(test_scores)

In [ ]:
for name, pipe in model_pipes_list:
    print(f"\n=== {name} ===")
    
    # Column names from that pipeline's own preprocess step
    ct = pipe.named_steps["preprocess"]
    feature_names = ct.get_feature_names_out()
    print("n_features (names):", len(feature_names))
    
    if name == "logit":
        est = pipe.named_steps["logit"]
        importances = est.coef_.ravel()
    elif name == "tree":
        est = pipe.named_steps["tree"]
        importances = est.feature_importances_
    elif name == "xgboost":
        # make sure this matches the actual step name in the pipe!
        est = pipe.named_steps["xgb"]
        importances = est.feature_importances_
    else:
        continue
    
    print("n_features (importances):", importances.shape[0])


In [ ]:
# feature importances per model

rows = []

for name, pipe in model_pipes_list:
    # get this pipeline's feature names
    ct = pipe.named_steps["preprocess"]
    feature_names = ct.get_feature_names_out()

    # get this pipeline's importances
    if name == "logit":
        est = pipe.named_steps["logit"]
        importances = est.coef_.ravel()
    elif name == "tree":
        est = pipe.named_steps["tree"]
        importances = est.feature_importances_
    elif name == "xgboost":
        est = pipe.named_steps["xgb"]
        importances = est.feature_importances_
    else:
        continue

    # sanity check
    assert len(feature_names) == importances.shape[0], f"{name} mismatch"

    rows.append(
        pd.DataFrame({
            "model": name,
            "feature": feature_names,
            "importance": importances,
            "abs_fi": np.abs(importances),
        })
    )

fi_all = pd.concat(rows, ignore_index=True)


In [ ]:
# logistic feature importances

rows =[]

for name, model in model_pipes_list:
    top_ten = fi_all[fi_all['model']==name].sort_values(by='abs_fi', ascending=False).head(10)
    
    display(top_ten)

In [ ]:
# plot predictions versus truth
def plot_classification_diagnostics_multi(models, X_test, y_test):
    """
    Plot ROC, Precision–Recall, and Calibration curves
    for multiple classifiers on the same row of subplots.
    """

    pos_rate = y_test.mean()

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))

    # =========================
    # 1. ROC curves
    # =========================
    for name, model in models:
        y_proba = model.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        roc_auc = auc(fpr, tpr)
        ax[0].plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.3f})")

    ax[0].plot([0, 1], [0, 1], "k--", label="Random")
    ax[0].set_xlabel("False Positive Rate")
    ax[0].set_ylabel("True Positive Rate")
    ax[0].set_title("ROC Curve — All Models")
    ax[0].legend(loc="lower right")
    ax[0].grid(alpha=0.3)

    # =========================
    # 2. Precision–Recall curves
    # =========================
    for name, model in models:
        y_proba = model.predict_proba(X_test)[:, 1]
        precision, recall, _ = precision_recall_curve(y_test, y_proba)
        ap = average_precision_score(y_test, y_proba)
        ax[1].plot(recall, precision, label=f"{name} (AP = {ap:.3f})")

    ax[1].hlines(pos_rate, 0, 1, linestyles="dashed",
                 label=f"Baseline = {pos_rate:.2f}")
    ax[1].set_xlabel("Recall")
    ax[1].set_ylabel("Precision")
    ax[1].set_title("Precision–Recall Curve — All Models")
    ax[1].legend(loc="lower right")
    ax[1].grid(alpha=0.3)

    # =========================
    # 3. Calibration curves
    # =========================
    for name, model in models:
        y_proba = model.predict_proba(X_test)[:, 1]
        prob_true, prob_pred = calibration_curve(y_test, y_proba, n_bins=10)
        ax[2].plot(prob_pred, prob_true, "o-", label=name)

    ax[2].plot([0, 1], [0, 1], "k--", label="Perfect calibration")
    ax[2].set_xlabel("Predicted probability")
    ax[2].set_ylabel("Observed frequency")
    ax[2].set_title("Calibration Curve — All Models")
    ax[2].legend()
    ax[2].grid(alpha=0.3)

    fig.tight_layout()
    plt.savefig('roc-pr-cal-curves.jpg')
    plt.show()


In [ ]:
# use function

plot_classification_diagnostics_multi(model_pipes_list, X_test, y_test)